In [5]:
# 케라스 콜백 사용 준비하기

from tensorflow.keras.datasets import mnist
import tensorflow as tf
import numpy as np

# 텐서플로우 저장소에서 데이터를 다운로드 받습니다.
(x_train, y_train), (x_test, y_test) = mnist.load_data(path = 'mnist.npz')

from sklearn.model_selection import train_test_split

# 훈련/검증 데이터를 얻기 위해 0.7/0.3의 비율로 분리합니다.
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                                  test_size=0.3, random_state=777)

num_x_train = x_train.shape[0]
num_x_val = x_val.shape[0]
num_x_test = x_test.shape[0]

# 모델의 입력으로 사용하기 위한 전처리 과정입니다.
x_train = (x_train.reshape(-1, 28, 28, 1))
x_val = (x_val.reshape(-1, 28, 28, 1))
x_test = (x_test.reshape(-1, 28, 28, 1))

ce_layer = tf.keras.layers.CategoryEncoding(num_tokens=np.unique(y_train).__len__(),
                                            output_mode="one_hot")

# 각 데이터의 레이블을 범주형 형태로 변경합니다.
y_train = ce_layer(list(y_train))
y_val = ce_layer(list(y_val))
y_test = ce_layer(list(y_test))

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.layers import Input

rescale_layer = tf.keras.layers.Rescaling(scale = 1./255)

# 함수형 API는 Input()을 통해 입력값의 형태를 정의해주어야 합니다.
inputs = Input(shape = (28, 28, 1))
x = rescale_layer(inputs)
x = Conv2D(32, (3, 3), activation = 'relu')(x)
x = Conv2D(32, (3, 3), activation = 'relu')(x)
x = MaxPooling2D(strides = 2)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(10, activation = 'softmax')(x)

# 위에서 정의한 층을 포함하고 있는 모델을 생성합니다.
model = Model(inputs = inputs, outputs = x)

model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['acc'])


In [8]:
# ModelCheckpoint 콜백 사용하기

from tensorflow.keras.callbacks import ModelCheckpoint

filepath = './best_model.keras'

# 콜백을 정의합니다.
callbacks = [ModelCheckpoint(filepath = filepath, monitor = 'val_loss', verbose = 1, save_best_only = True)]

# callbacks 인자를 통해 정의한 콜백을 전달합니다.
model.fit(x_train, y_train,
          batch_size = 32,
          validation_data = (x_val, y_val),
          epochs = 10,
          callbacks = callbacks)

Epoch 1/10
1312/1313 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.2701 - loss: 1.9731
Epoch 1: val_loss improved from inf to 1.40281, saving model to ./best_model.keras
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 30s 18ms/step - acc: 0.2703 - loss: 1.9727 - val_acc: 0.4817 - val_loss: 1.4028
Epoch 2/10
1310/1313 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.5323 - loss: 1.3227
Epoch 2: val_loss improved from 1.40281 to 1.15006, saving model to ./best_model.keras
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 24s 18ms/step - acc: 0.5324 - loss: 1.3225 - val_acc: 0.6158 - val_loss: 1.1501
Epoch 3/10
1310/1313 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.6486 - loss: 1.0883
Epoch 3: val_loss improved from 1.15006 to 0.95670, saving model to ./best_model.keras
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 24s 18ms/step - acc: 0.6487 - loss: 1.0882 - val_acc: 0.7031 - val_loss: 0.9567
Epoch 4/10
1310/1313 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.7140 - loss: 0.9245
Epoch 4: val_loss improved from 0.95670 to 0.81619, saving model to ./best

In [9]:
# EarlyStopping 콜백 사용하기

from tensorflow.keras.callbacks import EarlyStopping

# 콜백을 정의합니다.
callbacks = [EarlyStopping(monitor = 'val_loss', patience = 3, verbose = 1)]

# callbacks  인자를 통해 정의한 콜백을 전달합니다.
model.fit(x_train, y_train,
          batch_size = 32,
          validation_data = (x_val, y_val),
          epochs = 30,
          callbacks = callbacks)

Epoch 1/30
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - acc: 0.8744 - loss: 0.4093 - val_acc: 0.8768 - val_loss: 0.4126
Epoch 2/30
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - acc: 0.8816 - loss: 0.3983 - val_acc: 0.8816 - val_loss: 0.4007
Epoch 3/30
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - acc: 0.8824 - loss: 0.3871 - val_acc: 0.8914 - val_loss: 0.3727
Epoch 4/30
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - acc: 0.8854 - loss: 0.3720 - val_acc: 0.8913 - val_loss: 0.3601
Epoch 5/30
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 23s 17ms/step - acc: 0.8919 - loss: 0.3527 - val_acc: 0.9001 - val_loss: 0.3377
Epoch 6/30
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - acc: 0.8931 - loss: 0.3497 - val_acc: 0.8987 - val_loss: 0.3389
Epoch 7/30
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 23s 17ms/step - acc: 0.8984 - loss: 0.3389 - val_acc: 0.9094 - val_loss: 0.3052
Epoch 8/30
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - acc: 0.8996 - loss: 0.3263 - val_acc: 0.8882 - val_loss: 0.3510
Epoch 9/30
1313/1313 ━━━━━

In [11]:
# ReduceLROnPlataeu 콜백 사용하기
from tensorflow.keras.callbacks import ReduceLROnPlateau

# 콜백 정의하기
callbacks = [ReduceLROnPlateau(monitor = 'val_loss', patience = 3, factor = 0.2,
                               verbose = 1, min_lr = 1e-5)]

# callbacks 인자를 통해 정의한 콜백을 전달합니다.
model.fit(x_train, y_train, 
          batch_size = 32,
          validation_data = (x_val, y_val),
          epochs = 50,
          callbacks = callbacks)

Epoch 1/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 23s 17ms/step - acc: 0.9377 - loss: 0.2055 - val_acc: 0.9312 - val_loss: 0.2203 - learning_rate: 0.0010
Epoch 2/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - acc: 0.9381 - loss: 0.2009 - val_acc: 0.9350 - val_loss: 0.2084 - learning_rate: 0.0010
Epoch 3/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 23s 18ms/step - acc: 0.9387 - loss: 0.1971 - val_acc: 0.9391 - val_loss: 0.2006 - learning_rate: 0.0010
Epoch 4/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 23s 18ms/step - acc: 0.9385 - loss: 0.2001 - val_acc: 0.9282 - val_loss: 0.2265 - learning_rate: 0.0010
Epoch 5/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 23s 17ms/step - acc: 0.9403 - loss: 0.1929 - val_acc: 0.9371 - val_loss: 0.1979 - learning_rate: 0.0010
Epoch 6/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - acc: 0.9389 - loss: 0.1981 - val_acc: 0.9309 - val_loss: 0.2202 - learning_rate: 0.0010
Epoch 7/50
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - acc: 0.9411 - loss: 0.1895 - val_acc: 0.9414 - val_loss: 0.1873 - 

In [12]:
# TensorBoard 콜백 사용하기

from tensorflow.keras.callbacks import TensorBoard
from tensorboard.plugins.hparams import api as hp

logdir = './logs'

# 콜백을 정의합니다.
callbacks = [TensorBoard(log_dir = logdir, histogram_freq = 1,
                         write_graph = True, write_images = True),
                         hp.KerasCallback(logdir, {'num_filters':32})]

# callbacks 인자를 통해 정의한 콜백을 전달합니다.
model.fit(x_train, y_train,
          batch_size = 32,
          validation_data = (x_val, y_val),
          epochs = 1,
          callbacks = callbacks)

1313/1313 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - acc: 0.9514 - loss: 0.1586 - val_acc: 0.9489 - val_loss: 0.1640



In [15]:
# Tensorboard 실행
# auth, Refresh Error가 발생할 경우, 명령프롬프트 창에서 tensorboard dev upload --logdir ./logdir를 실행하고
# 기기를 연결한뒤 후에 나오는 링크로 실험 결과를 확인할 수 있습니다.
# Tensorboard dev는 public입니다. sensitive data 업로드는 경고됩니다.

!tensorboard dev upload --logdir ./logs \
--name "My test" \
--description "This is my first tensorboard"

'tensorboard'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [16]:
# 아래 명령어를 실행하면 tensorboard에 upload한 실험을 확인할 수 있습니다.
# 해당 링크를 따라가면 실험결과를 보여줍니다.
!tensorboard dev list

'tensorboard'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
